<a href="https://colab.research.google.com/github/lphuong304/CS116.M11/blob/main/Practice%20In%20Class/19520227/BaiTap9/19520227_BT9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"><strong>BÀI TẬP 8 - 26/10/2021</strong></h1>
<h1 align="center"><strong>LẬP TRÌNH PYTHON CHO MÁY HỌC CS116.M11 - BÀI TOÁN XÂY DỰNG MODULE </strong></h1>

# GIỚI THIỆU:

1. Họ và tên: Nguyễn Ngọc Lan Phương
2. MSSV: 19520227 - KHMT2019
3. Lớp: CS116.M11
4. Bài tập: Bài toán Xây dựng Module.
5. Ngày thực hiện: 2/11/2021

# MỤC LỤC
1. [Get data](#1)
2. [Import packages](#2)
3. [EDA](#3)
4. [Preprocessing Data](#4)
5. [Split train and test set](#5)
6. [Standarlize data](#s)
7. [Built module to train](#6)
8. [Evaluate](#7)
9. [Predict on Custom test](#8)
10. [Evaluate by KFold Cross Validation with K=5](#9) 

<a name = "1"></a>
#1. Get data 

In [ ]:
!gdown --id 17fb-FqlyeRwrW9l9uNl-qD1EprfRdgvC

Downloading...
From: https://drive.google.com/uc?id=17fb-FqlyeRwrW9l9uNl-qD1EprfRdgvC
To: /content/Churn_Modelling.csv
100% 685k/685k [00:00<00:00, 43.0MB/s]


<a name = "2"></a>
#2. Import Packages

In [ ]:
import numpy as np
import tensorflow as tf
import keras
import keras_preprocessing
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, KFold
from sklearn.compose import ColumnTransformer

<a name = "3"></a>
#3. EDA

In [ ]:
data = pd.read_csv('/content/Churn_Modelling.csv')
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [ ]:
data.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [ ]:
data.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

<a name = "4"></a>
#4. Preprocessing Data

##4.1. Drop unnecessay features

In [ ]:
del data['RowNumber']
del data['Surname']
del data['CustomerId']

## 4.2. Encode Caterogical Data

In [ ]:
lb = LabelEncoder()
data['Gender'] = lb.fit_transform(data['Gender'])
OneHotData = pd.get_dummies(data, columns = ['Geography'])

In [ ]:
OneHotData

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1,0,0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1,0,0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1,0,0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0,1,0


In [ ]:
OneHotData = OneHotData[['CreditScore',	'Gender',	'Age',	'Tenure',	'Balance',	'NumOfProducts',	'HasCrCard', 'IsActiveMember',	'EstimatedSalary',		'Geography_France', 'Geography_Germany', 'Geography_Spain' ,'Exited']]

In [ ]:
X = OneHotData.iloc[:,:-1]
y = OneHotData.iloc[:,-1]
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,1,0,0
9996,516,1,35,10,57369.61,1,1,1,101699.77,1,0,0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,0,0
9998,772,1,42,3,75075.31,2,1,0,92888.52,0,1,0


<a name = "5"></a>
#5. Split train and test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
print('X train shape:{}'.format(X_train.shape))
print('X test shape:{}'.format(X_test.shape))
print('y train shape:{}'.format(y_train.shape))
print('y test shape:{}'.format(y_test.shape))

X train shape:(8000, 12)
X test shape:(2000, 12)
y train shape:(8000,)
y test shape:(2000,)


In [ ]:
#reshape labels
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

<a name = "s"></a>
#6. Standarlize data

In [ ]:
scaler = StandardScaler()
scaledX_Train = scaler.fit_transform(X_train)
scaledX_Tesst = scaler.transform(X_test)
print('Scaled X train shape:{}'.format(X_train.shape))
print('Scaled X test shape:{}'.format(X_test.shape))

Scaled X train shape:(8000, 12)
Scaled X test shape:(2000, 12)


<a name = "6"></a>
#7. Training model

##7.1 Built model

In [ ]:
model = tf.keras.models.Sequential()

In [ ]:
model.add(tf.keras.layers.Flatten(input_shape = scaledX_Train[0].shape))
model.add(tf.keras.layers.Dense(units=6, activation='relu'))
model.add(tf.keras.layers.Dense(units=6, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

##7.2 Training

In [ ]:
model.fit(scaledX_Train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 4s 3ms/step - loss: 0.5953 - accuracy: 0.7209
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5067 - accuracy: 0.7945
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4734 - accuracy: 0.7945
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4535 - accuracy: 0.7944
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4391 - accuracy: 0.7975
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4297 - accuracy: 0.8065
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4220 - accuracy: 0.8171
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4149 - accuracy: 0.8266
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4092 - accuracy: 0.8329
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4043 - accura

<a name = "7"></a>
#8. Evaluate


##8.1 Predict

In [ ]:
y_pred = model.predict(scaledX_Tesst)
y_pred = (y_pred > 0.5)

##8.2 Evaluate

In [ ]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

       False       0.95      0.88      0.91      1724
        True       0.48      0.69      0.57       276

    accuracy                           0.86      2000
   macro avg       0.71      0.79      0.74      2000
weighted avg       0.88      0.86      0.87      2000



<a name ="9"></a>
#9. Preidict on custom dataset

In [ ]:
#Custom test:
# Geography: France
# Credit Score: 600
# Gender: Male
# Age: 40 years old
# Tenure: 3 years
# Balance: $ 60000
# No.of Products: 2
# Has credit card: Yes
# Active Member: Yes
# Estimated Salary: $50000

In [ ]:
def getTest(CreditScore, Gender,	Age,	Tenure,	Balance,	NumOfProducts,	HasCrCard,	IsActiveMember,	EstimatedSalary,	Geography_France,	Geography_Germany,	Geography_Spain):
  specific_customer = {'CreditScore':600,'Gender':Gender,'Age':Age,'Tenure':Tenure,'Balance':Balance,'NumOfProducts':NumOfProducts, 'HasCrCard':HasCrCard,'IsActiveMember':IsActiveMember,'EstimatedSalary':EstimatedSalary,'Geography_France':Geography_France,	'Geography_Germany':Geography_Germany,'Geography_Spain':Geography_Spain}
  test_customer = pd.DataFrame()
  test_customer = test_customer.append(specific_customer,ignore_index=True)
  test_customer = test_customer[['CreditScore',	'Gender',	'Age',	'Tenure',	'Balance',	'NumOfProducts',	'HasCrCard', 'IsActiveMember',	'EstimatedSalary',		'Geography_France', 'Geography_Germany', 'Geography_Spain']]
  return test_customer

In [ ]:
specific_customer =getTest(600,1,40,3,60000,2, 1,1,50000,1,0,0)
specific_customer

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600.0,1.0,40.0,3.0,60000.0,2.0,1.0,1.0,50000.0,1.0,0.0,0.0


In [ ]:
#scale custom test
scaled_test_customer = scaler.transform(specific_customer)
print(scaled_test_customer)

[[-0.53598516  0.91324755  0.10479359 -0.69539349 -0.25781119  0.80843615
   0.64920267  0.97481699 -0.87683221  1.00150113 -0.57946723 -0.57638802]]


In [ ]:
#predict
specific_customer = model.predict(scaled_test_customer)
specific_customer

array([[0.02198168]], dtype=float32)

In [ ]:
if specific_customer[0][0] > 0.5:
  print('Exited!')
else:
  print('Not Exited!')

Not Exited!


<a name = "9"></a>
#10. Evaluate by KFold Cross Validation with K=5

In [ ]:
k = 5
evaluate = KFold(n_splits=k, random_state=None)
model = model

acc_score = []

for train_index , test_index in evaluate.split(scaledX_Train):
    print(train_index)
    X_train , X_test = scaledX_Train[train_index],scaledX_Train[test_index]
    y_train , y_test = y[train_index] , y[test_index]
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape = X_train[0].shape))
    model.add(tf.keras.layers.Dense(units=6, activation='relu'))
    model.add(tf.keras.layers.Dense(units=6, activation='relu'))
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

    model.fit(X_train,y_train, batch_size = 32, epochs = 100)
    predictY = model.predict(X_test)
    
    acc = model.evaluate(X_test , y_test)
    acc_score.append(acc)
    

[1600 1601 1602 ... 7997 7998 7999]
Epoch 1/100
200/200 [==============================] - 1s 3ms/step - loss: 0.5741 - accuracy: 0.7803
Epoch 2/100
200/200 [==============================] - 1s 3ms/step - loss: 0.5186 - accuracy: 0.7955
Epoch 3/100
200/200 [==============================] - 1s 3ms/step - loss: 0.5137 - accuracy: 0.7955
Epoch 4/100
200/200 [==============================] - 1s 3ms/step - loss: 0.5113 - accuracy: 0.7955
Epoch 5/100
200/200 [==============================] - 1s 3ms/step - loss: 0.5096 - accuracy: 0.7955
Epoch 6/100
200/200 [==============================] - 1s 3ms/step - loss: 0.5084 - accuracy: 0.7955
Epoch 7/100
200/200 [==============================] - 1s 4ms/step - loss: 0.5077 - accuracy: 0.7955
Epoch 8/100
200/200 [==============================] - 1s 3ms/step - loss: 0.5068 - accuracy: 0.7955
Epoch 9/100
200/200 [==============================] - 1s 3ms/step - loss: 0.5061 - accuracy: 0.7955
Epoch 10/100
200/200 [==============================] -

In [ ]:
print('Mean accuravy:{}'.format(np.mean(acc_score)))

Mean accuravy:0.6543686747550964
